# Tugas klasifikasi teks

Dalam modul ini, kita akan memulai dengan tugas klasifikasi teks sederhana berdasarkan dataset **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: kita akan mengklasifikasikan judul berita ke dalam salah satu dari 4 kategori: Dunia, Olahraga, Bisnis, dan Sains/Teknologi.

## Dataset

Untuk memuat dataset, kita akan menggunakan API **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Kita sekarang dapat mengakses bagian pelatihan dan pengujian dari dataset dengan menggunakan `dataset['train']` dan `dataset['test']` masing-masing:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Mari cetak 10 berita utama baru pertama dari dataset kita:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Vektorisasi Teks

Sekarang kita perlu mengubah teks menjadi **angka** yang dapat direpresentasikan sebagai tensor. Jika kita ingin representasi pada tingkat kata, kita perlu melakukan dua hal:

* Menggunakan **tokenizer** untuk membagi teks menjadi **token**.
* Membuat **kosakata** dari token-token tersebut.

### Membatasi Ukuran Kosakata

Dalam contoh dataset AG News, ukuran kosakata cukup besar, lebih dari 100 ribu kata. Secara umum, kita tidak memerlukan kata-kata yang jarang muncul dalam teks — hanya beberapa kalimat yang akan memilikinya, dan model tidak akan belajar dari kata-kata tersebut. Oleh karena itu, masuk akal untuk membatasi ukuran kosakata ke angka yang lebih kecil dengan memberikan argumen pada konstruktor vektorisasi:

Kedua langkah tersebut dapat ditangani menggunakan lapisan **TextVectorization**. Mari kita buat objek vektorisasi, lalu panggil metode `adapt` untuk memproses semua teks dan membangun kosakata:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Catatan** bahwa kami hanya menggunakan sebagian kecil dari keseluruhan dataset untuk membangun kosakata. Kami melakukannya untuk mempercepat waktu eksekusi dan agar Anda tidak perlu menunggu terlalu lama. Namun, kami mengambil risiko bahwa beberapa kata dari keseluruhan dataset tidak akan dimasukkan ke dalam kosakata, dan akan diabaikan selama pelatihan. Oleh karena itu, menggunakan ukuran kosakata penuh dan menjalankan seluruh dataset selama `adapt` seharusnya dapat meningkatkan akurasi akhir, meskipun tidak secara signifikan.

Sekarang kita dapat mengakses kosakata yang sebenarnya:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Dengan menggunakan vectorizer, kita dapat dengan mudah mengkodekan teks apa pun ke dalam serangkaian angka:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Representasi teks Bag-of-words

Karena kata-kata mewakili makna, terkadang kita dapat memahami arti dari sebuah teks hanya dengan melihat kata-kata individualnya, tanpa memperhatikan urutannya dalam kalimat. Sebagai contoh, saat mengklasifikasikan berita, kata-kata seperti *cuaca* dan *salju* kemungkinan besar menunjukkan *ramalan cuaca*, sementara kata-kata seperti *saham* dan *dolar* akan mengarah pada *berita keuangan*.

Representasi vektor **Bag-of-words** (BoW) adalah representasi vektor tradisional yang paling sederhana untuk dipahami. Setiap kata dikaitkan dengan indeks vektor, dan elemen vektor berisi jumlah kemunculan setiap kata dalam dokumen tertentu.

![Gambar yang menunjukkan bagaimana representasi vektor bag-of-words disimpan dalam memori.](../../../../../translated_images/id/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Anda juga dapat memikirkan BoW sebagai jumlah dari semua vektor one-hot-encoded untuk setiap kata dalam teks.

Di bawah ini adalah contoh cara menghasilkan representasi bag-of-words menggunakan pustaka python Scikit Learn:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Kita juga dapat menggunakan vektorisasi Keras yang telah kita definisikan di atas, mengonversi setiap nomor kata menjadi one-hot encoding dan menjumlahkan semua vektor tersebut:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Catatan**: Anda mungkin terkejut bahwa hasilnya berbeda dari contoh sebelumnya. Alasannya adalah dalam contoh Keras, panjang vektor sesuai dengan ukuran kosakata, yang dibangun dari seluruh dataset AG News, sementara dalam contoh Scikit Learn, kami membangun kosakata dari teks sampel secara langsung.


## Melatih Klasifikasi BoW

Sekarang setelah kita mempelajari cara membangun representasi bag-of-words dari teks kita, mari kita latih sebuah klasifikasi yang menggunakannya. Pertama, kita perlu mengubah dataset kita menjadi representasi bag-of-words. Hal ini dapat dilakukan dengan menggunakan fungsi `map` seperti berikut:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Sekarang mari kita definisikan jaringan neural classifier sederhana yang mengandung satu lapisan linear. Ukuran input adalah `vocab_size`, dan ukuran output sesuai dengan jumlah kelas (4). Karena kita sedang menyelesaikan tugas klasifikasi, fungsi aktivasi akhirnya adalah **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Karena kita memiliki 4 kelas, akurasi di atas 80% adalah hasil yang baik.

## Melatih sebuah classifier sebagai satu jaringan

Karena vectorizer juga merupakan lapisan Keras, kita dapat mendefinisikan sebuah jaringan yang menyertakannya, dan melatihnya secara end-to-end. Dengan cara ini, kita tidak perlu memvektorisasi dataset menggunakan `map`, kita cukup memberikan dataset asli ke input jaringan.

> **Note**: Kita tetap harus menerapkan map pada dataset kita untuk mengubah bidang dari dictionary (seperti `title`, `description`, dan `label`) menjadi tuple. Namun, saat memuat data dari disk, kita dapat membangun dataset dengan struktur yang diperlukan sejak awal.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigram, Trigram, dan N-gram

Salah satu keterbatasan pendekatan bag-of-words adalah bahwa beberapa kata merupakan bagian dari ekspresi multi-kata. Sebagai contoh, kata 'hot dog' memiliki arti yang sepenuhnya berbeda dibandingkan dengan kata 'hot' dan 'dog' dalam konteks lain. Jika kita selalu merepresentasikan kata 'hot' dan 'dog' menggunakan vektor yang sama, hal ini dapat membingungkan model kita.

Untuk mengatasi masalah ini, **representasi n-gram** sering digunakan dalam metode klasifikasi dokumen, di mana frekuensi setiap kata, pasangan kata, atau tiga kata menjadi fitur yang berguna untuk melatih classifier. Dalam representasi bigram, misalnya, kita akan menambahkan semua pasangan kata ke dalam kosakata, selain kata-kata asli.

Di bawah ini adalah contoh cara menghasilkan representasi bigram bag-of-words menggunakan Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Kelemahan utama pendekatan n-gram adalah ukuran kosakata mulai tumbuh dengan sangat cepat. Dalam praktiknya, kita perlu menggabungkan representasi n-gram dengan teknik reduksi dimensi, seperti *embeddings*, yang akan kita bahas di unit berikutnya.

Untuk menggunakan representasi n-gram dalam dataset **AG News** kita, kita perlu meneruskan parameter `ngrams` ke konstruktor `TextVectorization` kita. Panjang kosakata bigram **jauh lebih besar**, dalam kasus kita lebih dari 1,3 juta token! Oleh karena itu, masuk akal untuk membatasi token bigram dengan jumlah yang wajar.

Kita bisa menggunakan kode yang sama seperti di atas untuk melatih classifier, namun, itu akan sangat tidak efisien dalam penggunaan memori. Di unit berikutnya, kita akan melatih classifier bigram menggunakan embeddings. Sementara itu, Anda dapat bereksperimen dengan pelatihan classifier bigram di notebook ini dan melihat apakah Anda bisa mendapatkan akurasi yang lebih tinggi.


## Menghitung Vektor BoW Secara Otomatis

Dalam contoh di atas, kita menghitung vektor BoW secara manual dengan menjumlahkan one-hot encoding dari masing-masing kata. Namun, versi terbaru TensorFlow memungkinkan kita untuk menghitung vektor BoW secara otomatis dengan memberikan parameter `output_mode='count'` ke konstruktor vectorizer. Hal ini membuat proses mendefinisikan dan melatih model kita menjadi jauh lebih mudah:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Frekuensi istilah - frekuensi dokumen terbalik (TF-IDF)

Dalam representasi BoW, kemunculan kata diberi bobot menggunakan teknik yang sama tanpa memandang kata itu sendiri. Namun, jelas bahwa kata-kata yang sering muncul seperti *a* dan *in* jauh kurang penting untuk klasifikasi dibandingkan istilah-istilah khusus. Dalam sebagian besar tugas NLP, beberapa kata lebih relevan daripada yang lain.

**TF-IDF** adalah singkatan dari **term frequency - inverse document frequency**. Ini adalah variasi dari bag-of-words, di mana alih-alih menggunakan nilai biner 0/1 untuk menunjukkan kemunculan kata dalam sebuah dokumen, digunakan nilai floating-point yang berkaitan dengan frekuensi kemunculan kata dalam korpus.

Secara formal, bobot $w_{ij}$ dari sebuah kata $i$ dalam dokumen $j$ didefinisikan sebagai:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
di mana
* $tf_{ij}$ adalah jumlah kemunculan $i$ dalam $j$, yaitu nilai BoW yang telah kita lihat sebelumnya
* $N$ adalah jumlah dokumen dalam koleksi
* $df_i$ adalah jumlah dokumen yang mengandung kata $i$ dalam seluruh koleksi

Nilai TF-IDF $w_{ij}$ meningkat secara proporsional dengan jumlah kemunculan kata dalam sebuah dokumen dan disesuaikan dengan jumlah dokumen dalam korpus yang mengandung kata tersebut, yang membantu mengoreksi fakta bahwa beberapa kata muncul lebih sering daripada yang lain. Sebagai contoh, jika sebuah kata muncul di *setiap* dokumen dalam koleksi, $df_i=N$, dan $w_{ij}=0$, maka istilah-istilah tersebut akan sepenuhnya diabaikan.

Anda dapat dengan mudah membuat vektorisasi TF-IDF dari teks menggunakan Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Dalam Keras, lapisan `TextVectorization` dapat secara otomatis menghitung frekuensi TF-IDF dengan melewatkan parameter `output_mode='tf-idf'`. Mari ulangi kode yang kita gunakan di atas untuk melihat apakah menggunakan TF-IDF meningkatkan akurasi:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Kesimpulan

Meskipun representasi TF-IDF memberikan bobot frekuensi pada kata-kata yang berbeda, mereka tidak mampu merepresentasikan makna atau urutan. Seperti yang dikatakan oleh ahli linguistik terkenal J. R. Firth pada tahun 1935, "Makna lengkap dari sebuah kata selalu bersifat kontekstual, dan tidak ada studi tentang makna yang terlepas dari konteks yang dapat dianggap serius." Kita akan mempelajari cara menangkap informasi kontekstual dari teks menggunakan pemodelan bahasa di bagian selanjutnya dari kursus ini.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
